In [2]:
import sys
import math
import csv
import random
import json 
import time

try:
    jparams = json.load(open('params.json'))
except:
    print("ERROR: something is wrong with the params.json file.")
    sys.exit()
    
#-- store the input 3D points in list
list_pts_3d = []
with open(jparams['input-file']) as csvfile:
    r = csv.reader(csvfile, delimiter=' ')
    header = next(r)
    for line in r:
        p = list(map(float, line)) #-- convert each str to a float
        assert(len(p) == 3)
        list_pts_3d.append(p)

In [4]:
import math
import numpy
import scipy.spatial

In [9]:
class BoundingBox:
    """A 2D bounding box"""
    
    def __init__(self, points):
        if len(points) == 0:
            raise ValueError("Can't compute bounding box of empty list")
        
        self.minx, self.miny = float("inf"), float("inf")
        self.maxx, self.maxy = float("-inf"), float("-inf")
        for x, y, *_ in points:
            # Set min coords
            if x < self.minx:
                self.minx = x
            if y < self.miny:
                self.miny = y
            # Set max coords
            if x > self.maxx:
                self.maxx = x
            elif y > self.maxy:
                self.maxy = y
                
    @property
    def width(self):
        return self.maxx - self.minx
    @property
    def height(self):
        return self.maxy - self.miny
    def __repr__(self):
        return "BoundingBox(X: {} - {}, Y: {} - {})".format(
            self.minx, self.maxx, self.miny, self.maxy)
      
bbox = BoundingBox(list_pts_3d)
bbox

BoundingBox(X: 2.0 - 252.0, Y: 3.0 - 253.0)

In [5]:
class Raster:
  """Simple raster based on ESRI ASCII schema"""
  
  def __init__(self, bbox, cell_size, no_data=-9999):
    self.ncols = bbox.width // cell_size + 1
    self.nrows = bbox.height // cell_size + 1
    self.xllcenter = bbox.minx
    self.yllcenter = bbox.miny
    self.cell_size = cell_size
    self.no_data = no_data
    
    # initialize list of values with no_data
    self.values = [self.no_data] * self.ncols * self.nrows
    
    # initialize list of pixel center coords
    self.coords = []
    for i in range(self.ncols):
      for j in range(self.nrows):
        x = self.xllcenter + i * self.cell_size
        y = self.yllcenter + j * self.cell_size
        self.coords.append([x,y])
    
  def to_ascii(self):
    rows = [
      "NCOLS %d" % self.ncols,
      "NROWS %d" % self.nrows,
      "XLLCENTER %s" % self.xllcenter,
      "YLLCENTER %s" % self.yllcenter,
      "CELLSIZE %s" % self.cell_size,
      "NODATA_VALUE %s" % self.no_data,
      ' '.join(self.values)
    ]
    return '\n'.join(rows)

In [13]:
[[-9999]*3]*3

[[-9999, -9999, -9999], [-9999, -9999, -9999], [-9999, -9999, -9999]]